In [1]:
import logging
import datetime
import time
#from sitemap_parser import SitemapParser
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, NoSuchAttributeException, InvalidSelectorException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import codecs
import re
import requests
import json
import pprint as pprint

In [ ]:
def kitchensanctuary_crawler(self, path):
        """
        :param urls: Urls to parse.
        :param url_id: url_id from input table.
        :param website_id: site_id from input table.
        :param website_name: site_name.
        :return: Data points from website.
        """
        start_datetime = datetime.datetime.utcnow()
        path= 'C:/Users/Dell/Downloads/smart-cookie/kitchensanctuary/'
        files = os.listdir(path)
        count = 0
        for i in range(len(files)):
            count = count + 1
            logger.info('Loop Count %d, DF index Count %d', count, len(self.buffer_data_df.index))
            files[i] = path + '/' + files[i]
            file = codecs.open(files[i], "r", "utf-8")
            soup = BeautifulSoup(file.read(), features="lxml")
            info_container = soup.select_one('div.wprm-recipe-container')
            if info_container:
                content_series = self.get_empty_content_series()
                content_series["crawl_timestamp"] = datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
                content_series["url_id"] = self.site_doc['url_id']
                content_series["website_id"] = self.site_doc['site_id']
                content_series["website_name"] = self.site_doc['site_name']

                try:
                    title = soup.select_one('h2.wprm-recipe-name')
                    content_series['recipe_title'] = title.text
                except:
                    content_series['recipe_title'] = ""    

                try:
                    content_series['recipe_url'] = soup.select_one('meta[property="og:url"]')['content']
                except:
                    content_series['recipe_url'] = ""    

                try:
                    author_name = soup.select_one('span.wprm-recipe-details.wprm-recipe-author.wprm-block-text-bold').text
                    content_series['author_name'] = author_name
                except:
                    content_series['author_name']= ''
                    
                try:
                    try:
                        value = soup.select_one('span.wprm-recipe-servings.wprm-recipe-details.wprm-recipe-servings').text
                    except:
                        value = ""
                    try:
                        unit = soup.select_one('span.wprm-recipe-servings-unit.wprm-recipe-details-unit.wprm-block-text-normal').text
                    except:
                        unit = ""
                    serving_size = {'value': value, 'unit': unit}
                    content_series['serving_size'] = serving_size
                except:
                    content_series['serving_size'] = ""       

                try:
                    modified_date = soup.select_one('meta[property="article:modified_time"]')
                    content_series['modified_date'] = modified_date['content'].split('T')[0]
                except:
                    content_series['modified_date'] = ""

                try:
                    published_date = soup.select_one('meta[property="article:published_time"]')
                    content_series['published_date'] = published_date['content'].split('T')[0]
                except:
                    content_series['published_date'] = "" 
                
                try:
                    cuisine = info_container.select_one("span.wprm-recipe-cuisine")
                    if cuisine:
                        content_series['cuisine'] = cuisine.text
                except:
                    content_series['cuisine'] = ""
                
                try:
                    meal_course = soup.select_one("span.wprm-recipe-course")
                    if meal_course:
                        content_series['meal_course'] = meal_course.text
                except:
                    content_series['meal_course'] = ""

                time_data = {'hours': 0, 'minutes': 0, 'days': 0}
                try:
                    try:
                        preperation_time_minutes = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-minutes.wprm-recipe-prep_time.wprm-recipe-prep_time-minutes")
                        if preperation_time_minutes:
                            time_data['minutes'] = preperation_time_minutes.text
                    except:
                        time_data['minutes'] = "0"
                    try:
                        preperation_time_hours = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-hours.wprm-recipe-prep_time.wprm-recipe-prep_time-hours")
                        if preperation_time_hours:
                            time_data['hours'] = preperation_time_hours.text
                    except:
                        time_data['hours'] = "0"
                    try:
                        preperation_time_days = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-days.wprm-recipe-prep_time.wprm-recipe-prep_time-days")
                        if preperation_time_days:
                            time_data['days'] = preperation_time_days.text
                    except:
                        time_data['days'] = "0"
                    content_series['preperation_time'] = int(time_data['minutes'])+int(time_data['hours'])*60+int(time_data['days'])*24*60
                except:
                    content_series['preperation_time'] = ""

                time_data = {'hours': 0, 'minutes': 0, 'days': 0}
                try:
                    try:
                        cooking_time_minutes = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-minutes.wprm-recipe-cook_time.wprm-recipe-cook_time-minutes")
                        if cooking_time_minutes:
                            time_data['minutes'] = cooking_time_minutes.text
                    except:
                        time_data['minutes'] = "0"
                    try:
                        cooking_time_hours = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-hours.wprm-recipe-cook_time.wprm-recipe-cook_time-hours")
                        if cooking_time_hours:
                            time_data['hours'] = cooking_time_hours.text
                    except:
                        time_data['hours'] = "0"
                    try:
                        cooking_time_days = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-days.wprm-recipe-cook_time.wprm-recipe-cook_time-days")
                        if cooking_time_days:
                            time_data['days'] = cooking_time_days.text
                    except:
                        time_data['days'] = "0"
                    content_series['cooking_time'] = int(time_data['minutes'])+int(time_data['hours'])*60+int(time_data['days'])*24*60
                except:
                    content_series['cooking_time'] = ""

                time_data = {'hours': 0, 'minutes': 0, 'days': 0}
                try:
                    try:
                        total_time_minutes = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-minutes.wprm-recipe-total_time.wprm-recipe-total_time-minutes")
                        if total_time_minutes:
                            time_data['minutes'] = total_time_minutes.text
                    except:
                        time_data['minutes'] = "0"
                    try:
                        total_time_hours = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-hours.wprm-recipe-total_time.wprm-recipe-total_time-hours")
                        if total_time_hours:
                            time_data['hours'] = total_time_hours.text
                    except:
                        time_data['hours'] = "0"
                    try:
                        total_time_days = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-days.wprm-recipe-total_time.wprm-recipe-total_time-days")
                        if total_time_days:
                            time_data['days'] = total_time_days.text
                    except:
                        time_data['days'] = "0"
                    content_series['total_time'] = int(time_data['minutes'])+int(time_data['hours'])*60+int(time_data['days'])*24*60
                except:
                    content_series['total_time'] = ""

                
                try:
                    image = soup.select_one('meta[property="og:image"]')['content']
                    images_dict = {'image':image, 'title': '', 'caption': ''}
                    content_series["img_urls"] = [images_dict]
                except:
                    content_series['img_urls'] = []
                
                try:
                    description = soup.select_one('div.wprm-recipe-summary.wprm-block-text-normal').text
                    content_series['recipe_description'] = description
                except:
                    pass
                    
                ingredients_lst = []
                ingredients_container = soup.select_one(
                    "div.wprm-recipe-ingredients-container")
                if ingredients_container:
                    group_sections = ingredients_container.select(
                        'div.wprm-recipe-ingredient-group')
                    for group_section in group_sections:
                        section_name = ''
                        ingredients = []
                        section_tag = group_section.select_one("h4.wprm-recipe-group-name")
                        if section_tag:
                            section_name = section_tag.text
                        ingredient_tags = group_section.select(
                            'ul.wprm-recipe-ingredients>li.wprm-recipe-ingredient')
                        for ingredient in ingredient_tags:
                            ingredients.append(ingredient.text.strip("▢"))
                        ingredients_lst.append(
                            {'section': section_name, 'ingredients': ingredients})
                content_series["recipe_ingredient_list"] = ingredients_lst

                instructions_list = []
                title = ''
                instructions = soup.select('div.wprm-recipe-instruction-group>ul>li')
                instruction_lst = []
                for inst in instructions:
                    isnt_data = {'text': '', 'title': '', 'image': '', 'caption': ''}
                    isnt_data['text'] = inst.text
                    instruction_lst.append(isnt_data)
                instructions_list.append({'section': title, 'instructions': instruction_lst})
                content_series["recipe_instructions"] = instructions_list
                instruction_container = soup.select_one('div.wprm-recipe-instruction-group')
                if instruction_container:
                    content_series["recipe_instructions_html"] = str(instruction_container)
                
                self.parsed_data_df = self.parsed_data_df.append(content_series, ignore_index=True)
                self.buffer_data_df = self.buffer_data_df.append(content_series, ignore_index=True)
                if len(self.buffer_data_df.index) % 10 == 0:
                    self.buffer_data_df = self.buffer_data_df.replace({np.nan: None})
                    self.parsed_data_df = self.parsed_data_df.replace({np.nan: None})
                    recipe_content = self.buffer_data_df.to_dict('records')
                    recipe_urls = self.buffer_data_df[['recipe_url']].rename(
                        columns={'recipe_url': 'url'}).to_dict('records')
                    self._store_to_db(recipe_content, recipe_urls)
                    self.parsed_data_df.to_csv(self.site_doc['site_name'] + '.csv', index=False)
                    logger_stats.info('Start Time: %s, Total Dumps: %d, Elapsed Time: %s',
                                    start_datetime,
                                    len(self.buffer_data_df.index),
                                    datetime.datetime.utcnow() - start_datetime)
                    self.buffer_data_df.drop(self.buffer_data_df.index, inplace=True)
        if len(self.buffer_data_df.index):
            self.buffer_data_df = self.buffer_data_df.replace({np.nan: None})
            recipe_content = self.buffer_data_df.to_dict('records')
            recipe_url = self.buffer_data_df[['recipe_url']].rename(
                columns={'recipe_url': 'url'}).to_dict('records')
            self._store_to_db(recipe_content, recipe_url)
            self.parsed_data_df = self.parsed_data_df.replace({np.nan: None})
            self.parsed_data_df.to_csv(self.site_doc['site_name'] + '.csv', index=False)
        logger_stats.info('Start Time: %s, Total Dumps: %d, Elapsed Time: %s',
                        start_datetime,
                        len(self.buffer_data_df.index),
                        datetime.datetime.utcnow() - start_datetime)

In [2]:
url = 'https://www.kitchensanctuary.com/chocolate-almond-banana-bread/#recipe'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml') 

In [3]:
title = soup.select_one('h2.wprm-recipe-name')
title.text

'Chocolate Almond Banana Bread'

In [4]:
recipe_url = soup.select_one('meta[property="og:url"]')['content']
recipe_url

'https://www.kitchensanctuary.com/chocolate-almond-banana-bread/'

In [7]:
author_name = soup.select_one('span.wprm-recipe-details.wprm-recipe-author.wprm-block-text-bold').text
author_name


'Nicky Corbishley'

In [8]:
value = soup.select_one('span.wprm-recipe-servings.wprm-recipe-details.wprm-recipe-servings').text
value

'12'

In [9]:
unit = soup.select_one('span.wprm-recipe-servings-unit.wprm-recipe-details-unit.wprm-block-text-normal').text
unit

'slices'

In [10]:
serving_size = {'value': value, 'unit': unit}
serving_size

{'value': '12', 'unit': 'slices'}

In [11]:
modified_date = soup.select_one('meta[property="article:modified_time"]')
modified_date = modified_date['content'].split('T')[0]
modified_date


'2018-08-01'

In [12]:
published_date = soup.select_one('meta[property="article:published_time"]')
published_date = published_date['content'].split('T')[0]
published_date


'2014-11-04'

In [13]:
preperation_time_minutes = soup.select_one("span.wprm-recipe-details.wprm-recipe-details-minutes.wprm-recipe-prep_time.wprm-recipe-prep_time-minutes")
preperation_time_minutes.text


'20'

In [14]:
cooking_time_minutes = soup.select_one("span.wprm-recipe-details.wprm-recipe-details-minutes.wprm-recipe-cook_time.wprm-recipe-cook_time-minutes")
cooking_time_minutes.text

'55'

In [15]:
 total_time_minutes = soup.select_one("span.wprm-recipe-details.wprm-recipe-details-minutes.wprm-recipe-total_time.wprm-recipe-total_time-minutes")
total_time_minutes.text

'15'

In [17]:
total_time_hours = soup.select_one("span.wprm-recipe-details.wprm-recipe-details-hours.wprm-recipe-total_time.wprm-recipe-total_time-hours")

In [19]:
total_time_hours.text


'1'

In [20]:
image = soup.select_one('meta[property="og:image"]')['content']
images_dict = {'image':image, 'title': '', 'caption': ''}
img_url = [images_dict]
img_url

[{'image': 'https://www.kitchensanctuary.com/wp-content/uploads/2014/11/Chocolate-almond-banana-bread-square-2.jpg',
  'title': '',
  'caption': ''}]

In [21]:
description = soup.select_one('meta[property="og:description"]')['content']
description

'My\xa0Chocolate Almond Banana Bread is really delicious.\xa0Moist, chocolatey banana bread, dotted with swirls of almond frangipane, topped with creamy almond frosting and sprinkled with handfuls of crunchy candied almonds and pecans.'

In [25]:
ingredients_lst = []
ingredients = []
section_text = ''
lists = soup.select("div.wprm-recipe-ingredient-group>ul>li")
for li in lists:
    ingredients.append(li.text.strip("▢"))
ingredients_lst.append({'section': section_text, 'ingredients': ingredients})
recipe_ingredient_list = ingredients_lst
recipe_ingredient_list

[{'section': '',
  'ingredients': [' 60 g 2oz unsalted butter - softened',
   ' 50 g 1.75oz golden caster sugar',
   ' 1 large egg',
   ' 35 g 4tbsp plain flour',
   ' ¼ tsp baking powder',
   ' 60 g 2oz ground almonds',
   ' ½ tsp almond extract',
   ' 1 large egg',
   ' 75 ml 2.5fl oz vegetable oil',
   ' 125 g 4.5oz golden caster sugar',
   ' 3 very ripe bananas - peeled and roughly chopped',
   ' 150 g 5.3oz plain flour',
   ' 25 g 0.9oz cocoa powder - sieved',
   ' ½ tsp bicarbonate of soda',
   ' ½ tsp baking powder',
   ' 50 g 1.75oz chocolate chips',
   ' 140 g 1cup confectioners sugar - sieved',
   ' 2-3 tbsp. double/heavy cream',
   ' ¼ tsp vanilla extract',
   ' 1 tsp almond extract',
   ' Candied nuts',
   ' 120 g ½ cup golden caster sugar',
   ' 100 g ¾ cup mixed, whole pecans and almonds (skin on)']}]

In [32]:
ingredients_lst = []
ingredients_container = soup.select_one(
    "div.wprm-recipe-ingredients-container")
if ingredients_container:
    group_sections = ingredients_container.select(
        'div.wprm-recipe-ingredient-group')
    for group_section in group_sections:
        section_name = ''
        ingredients = []
        section_tag = group_section.select_one("h4.wprm-recipe-group-name")
        if section_tag:
            section_name = section_tag.text
        ingredient_tags = group_section.select(
            'ul.wprm-recipe-ingredients>li.wprm-recipe-ingredient')
        for ingredient in ingredient_tags:
            ingredients.append(ingredient.text.strip("▢"))
        ingredients_lst.append(
            {'section': section_name, 'ingredients': ingredients})
    recipe_ingredient_list = ingredients_lst

In [33]:
ingredients_lst

[{'section': 'Frangipane:',
  'ingredients': [' 60 g 2oz unsalted butter - softened',
   ' 50 g 1.75oz golden caster sugar',
   ' 1 large egg',
   ' 35 g 4tbsp plain flour',
   ' ¼ tsp baking powder',
   ' 60 g 2oz ground almonds',
   ' ½ tsp almond extract']},
 {'section': 'Chocolate Banana Loaf:',
  'ingredients': [' 1 large egg',
   ' 75 ml 2.5fl oz vegetable oil',
   ' 125 g 4.5oz golden caster sugar',
   ' 3 very ripe bananas - peeled and roughly chopped',
   ' 150 g 5.3oz plain flour',
   ' 25 g 0.9oz cocoa powder - sieved',
   ' ½ tsp bicarbonate of soda',
   ' ½ tsp baking powder',
   ' 50 g 1.75oz chocolate chips']},
 {'section': 'Almond Frosting:',
  'ingredients': [' 140 g 1cup confectioners sugar - sieved',
   ' 2-3 tbsp. double/heavy cream',
   ' ¼ tsp vanilla extract',
   ' 1 tsp almond extract',
   ' Candied nuts',
   ' 120 g ½ cup golden caster sugar',
   ' 100 g ¾ cup mixed, whole pecans and almonds (skin on)']}]

In [34]:
instructions_list = []
title = ''
instructions = soup.select('div.wprm-recipe-instruction-group>ul>li')
instruction_lst = []
for inst in instructions:
    isnt_data = {'text': '', 'title': '', 'image': '', 'caption': ''}
    isnt_data['text'] = inst.text
    instruction_lst.append(isnt_data)
instructions_list.append({'section': title, 'instructions': instruction_lst})
recipe_instructions = instructions_list
recipe_instructions

[{'section': '',
  'instructions': [{'text': 'Preheat oven to 180c. Line a 2lb loaf tin with baking parchment.',
    'title': '',
    'image': '',
    'caption': ''},
   {'text': 'First make your frangipane. Beat the butter and sugar together for a few minutes until light and creamy (you can do this in a mixer or by hand). Add the egg and beat again for a minute. Add the flour, baking powder, ground almonds and almond extract and mix for 2 more minutes. Cover and place in the fridge whilst you make the chocolate banana cake.',
    'title': '',
    'image': '',
    'caption': ''},
   {'text': 'In a mixer, whisk the egg, oil and sugar until combined, then add in the bananas. Whisk again until the banana breaks down and you’ve got a slightly lumpy liquid.',
    'title': '',
    'image': '',
    'caption': ''},
   {'text': 'Add in the flour, sieved cocoa powder, baking powder and bicarbonate of soda. Gently stir into the banana mixture using a silicone spatula or wooden spoon until combine

In [35]:
cuisine = soup.select_one("span.wprm-recipe-cuisine").text
cuisine

'British'

In [36]:
meal_course = soup.select_one("span.wprm-recipe-course")
meal_course.text

'Cakes'

In [37]:
des = soup.select_one('div.wprm-recipe-summary.wprm-block-text-normal')
des.text

'Chocolate almond banana bread - Rich and moist chocolate banana bread, dotted with swirls of frangipane and topped with almond frosting and candied nuts.'

In [38]:
info_container = soup.select_one('div.wprm-recipe-container')
info_container

<div class="wprm-recipe-container" data-recipe-id="11198" data-servings="12" id="wprm-recipe-container-11198"><div class="wprm-recipe wprm-recipe-template-kschic"><div class="wprm-container-float-right">
<div class="wprm-recipe-image wprm-block-image-circle"><img alt="" class="attachment-250x250 size-250x250" data-lazy-sizes="(max-width: 250px) 100vw, 250px" data-lazy-src="https://www.kitchensanctuary.com/wp-content/uploads/2014/11/Chocolate-almond-banana-bread-square-2-300x300-1-300x300.jpg" data-lazy-srcset="https://www.kitchensanctuary.com/wp-content/uploads/2014/11/Chocolate-almond-banana-bread-square-2-300x300-1.jpg 300w, https://www.kitchensanctuary.com/wp-content/uploads/2014/11/Chocolate-almond-banana-bread-square-2-300x300-1-150x150.jpg 150w, https://www.kitchensanctuary.com/wp-content/uploads/2014/11/Chocolate-almond-banana-bread-square-2-300x300-1-180x180.jpg 180w, https://www.kitchensanctuary.com/wp-content/uploads/2014/11/Chocolate-almond-banana-bread-square-2-300x300-1-50